In [1]:
# import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import yaml

from typing import Dict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

%load_ext kedro.ipython
device = torch.device('mps')

[01/03/24 19:20:17] INFO     Resolved project path as: /Users/kpt/Code/kedro/gas-sensor-ml.         ]8;id=625959;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=528542;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[01/03/24 19:20:17] WARNING  /Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site- ]8;id=873315;file:///Users/kpt/.pyenv/versions/3.10.13/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=941550;file:///Users/kpt/.pyenv/versions/3.10.13/lib/python3.10/warnings.py#109\109]8;;\
                             packages/lazy_loader/__init__.py:78: KedroDeprecationWarning:                         
                             'CSVDataSet' has been renamed to 'CSVDataset', and the alias will be                  
                             removed in Kedro-Datasets 2.0.0                                                       
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    WARNING  /Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site- ]8;id=411973;file:///Users/kpt/.pyenv/versions/3.10.13/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=698074;file:///Users/kpt/.pyenv/versions/3.10.13/lib/python3.10/warnings.py#109\109]8;;\
                             packages/lazy_loader/__init__.py:78: KedroDeprecationWarning:                         
                             'PickleDataSet' has been renamed to 'PickleDataset', and the alias                    
                             will be removed in Kedro-Datasets 2.0.0                                               
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    INFO     Kedro project gas-sensor-ml                                            ]8;id=689852;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=457910;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=951619;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=193039;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

[01/03/24 19:20:18] INFO     Registered line magic 'run_viz'                                        ]8;id=556909;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=390319;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/ipython/__init__.py#115\115]8;;\

In [6]:
# Load model input which is already a df

model_input = catalog.load("model_input_table")
model_input.head()

[01/03/24 19:23:33] INFO     Loading data from 'model_input_table' (CSVDataset)...              ]8;id=494998;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=7921;file:///Users/kpt/.pyenv/versions/3.10.13/envs/kedro-env/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8,bin_9,...,bin_141,bin_142,bin_143,bin_144,bin_145,bin_146,bin_147,bin_148,bin_149,res_ratio
0,984.333333,984.0,983.333333,984.0,984.333333,982.0,983.500000,984.000000,983.5,982.000000,...,956.333333,959.0,957.333333,957.500000,957.666667,958.000000,959.5,960.000000,960.000000,1.451877
1,987.666667,987.0,986.666667,986.0,984.666667,986.5,985.333333,983.000000,985.0,985.000000,...,954.000000,955.0,953.666667,955.500000,954.666667,954.500000,955.0,953.333333,953.000000,1.344259
2,975.666667,973.5,975.666667,975.0,974.333333,974.5,974.000000,973.666667,973.5,972.666667,...,948.666667,948.5,947.000000,946.666667,948.000000,947.333333,947.0,946.000000,946.666667,1.337048
3,977.666667,977.5,978.000000,977.0,976.333333,976.5,976.666667,976.500000,976.0,975.000000,...,942.000000,942.0,943.000000,941.500000,941.500000,942.000000,942.0,943.000000,943.000000,1.341596
4,971.000000,970.5,971.000000,970.0,971.000000,970.0,970.000000,970.000000,969.0,968.666667,...,939.500000,941.0,941.000000,940.000000,940.666667,940.000000,940.0,940.500000,941.000000,1.345378


In [9]:
# check for nan in the data
model_input.isna().sum()
# model_input.shape


bin_0          0
bin_1        222
bin_2        243
bin_3        250
bin_4        289
            ... 
bin_146      261
bin_147      273
bin_148      248
bin_149      248
res_ratio      0
Length: 151, dtype: int64